In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
# 改动: padding=1 -> padding=0
class DoubleConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(out_ch, out_ch, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
        )
    def forward(self, x):
        return self.double_conv(x)
    
class OutConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv = nn.Conv2d(in_ch, out_ch, kernel_size=1)
    def forward(self, x):
        return self.conv(x)

In [5]:
class Down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),  
            DoubleConv(in_ch, out_ch)  
        )
    def forward(self, x):
        return self.maxpool_conv(x)
    
class Up(nn.Module):
    def __init__(self, in_ch, out_ch, bilinear=True):
        super().__init__()
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_ch // 2, in_ch // 2, kernel_size=2, stride=2)

        self.conv = DoubleConv(in_ch, out_ch)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # 对上采样后的特征图进行padding以适应拼接
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]
        
        #与原版不同, 这里是x1适应x2的大小, x1的尺寸变大了
        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2, diffY // 2, diffY - diffY // 2])
        
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)

In [6]:
class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True):
        super().__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 1024)

        self.up1 = Up(1024 + 512, 512, bilinear)
        self.up2 = Up(512 + 256, 256, bilinear)
        self.up3 = Up(256 + 128, 128, bilinear)
        self.up4 = Up(128 + 64, 64, bilinear)

        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        print(x.shape)
        
        x1 = self.inc(x)     # 64
        print(x1.shape)
        
        x2 = self.down1(x1)  # 128
        print(x2.shape)
        
        x3 = self.down2(x2)  # 256
        print(x3.shape)
        
        x4 = self.down3(x3)  # 512
        print(x4.shape)
        
        x5 = self.down4(x4)  # 1024
        print(x5.shape)
        print("===="*20)

        x = self.up1(x5, x4) # 512
        print(x5.shape, x4.shape)
        
        x = self.up2(x, x3)  # 256
        print(x.shape, x3.shape)

        x = self.up3(x, x2)  # 128
        print(x.shape, x1.shape)
        
        x = self.up4(x, x1)  # 64
        print(x.shape, x1.shape)
        
        logits = self.outc(x)
        return logits

model = UNet(n_channels=1, n_classes=1)  # 单通道输入，单通道输出（二分类）
input_tensor = torch.randn((1, 1, 572, 572))  # 示例输入（B, C, H, W）
output = model(input_tensor)

print(output.shape)   

torch.Size([1, 1, 572, 572])
torch.Size([1, 64, 572, 572])
torch.Size([1, 128, 286, 286])
torch.Size([1, 256, 143, 143])
torch.Size([1, 512, 71, 71])
torch.Size([1, 1024, 35, 35])
torch.Size([1, 1024, 35, 35]) torch.Size([1, 512, 71, 71])
torch.Size([1, 256, 143, 143]) torch.Size([1, 256, 143, 143])
torch.Size([1, 128, 286, 286]) torch.Size([1, 64, 572, 572])
torch.Size([1, 64, 572, 572]) torch.Size([1, 64, 572, 572])
torch.Size([1, 1, 572, 572])
